# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy.dialects.mssql.information_schema import columns

# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
# 实例化类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()
pd.set_option('display.max_columns', None)

# 定义数据库连接

In [3]:
def query(sql,
        host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
        user="wxz",
        password="5JRcY9SaiepVlIq7iuPo",
        database = '' ,
        port=3306
        ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips   
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method, tor.update_time, tomt.reason, pa.type, om.order_type, tp.classify_id
from  db_digua_business.t_order  om 
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 商品表
left join db_digua_business.t_product tp on tp.id=tod.product_id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )               -- 近15天数据
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-06'
;
'''

df_order = query(sql1)
df_order.shape

(378369, 40)

In [5]:
sql_risk = ''' -- risk等级表
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r from db_credit.risk
'''
df_risk = query(sql_risk)
# df_risk.loc[:, 'time_r'] = pd.to_datetime(df_risk.time.dt.date,errors="coerce")
# df_risk = df_risk.sort_values(by='time', ascending=False).groupby('id_card_r').head(1)

In [6]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
# df_re = df_re.sort_values(by='time', ascending=False).groupby('id_card_re').head(1)

In [7]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

# 合并数据

In [8]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(378369)

# 处理日期

In [9]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["下单日期"].dt.strftime('%Y-%m')
df.shape

(378369, 43)

In [10]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

# 处理备注信息

In [11]:
df = df[df['sku_attributes'].notnull()]

In [12]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  


def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [13]:
df['免审'] = np.where((df.is_vip=='1')&(df.status_result=='0'),1,0)

# 归属渠道

In [14]:

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

# 订单去重

In [15]:
df = clean.order_drop_duplicates(df)

# 定义状态

In [16]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')

# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)

# 导入服务订单

In [17]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-07-14'

In [18]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(46509, 68)

In [19]:
# 获取各个节点的状态
df = clean.status_node(df)

# 剔除商家信息

In [20]:
# 保留商家信息
df_merchant = df.copy()

In [21]:
# 剔除商家数据
df = clean.drop_merchant(df)

# 导入出库台账数据

In [22]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025")   #,skiprows=0 header 
df_ck.shape

(23998, 18)

In [23]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape

(4266, 101)

In [24]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
base_date = pd.Timestamp('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')

In [25]:
#按下单日期看每日各渠道出库单数
df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)

# 剔除据量数据

In [26]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)

# 总体

In [27]:
#总体
df_group_weekday = all_models.data_group(df, df2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
weekly_sum = week_models.week_data_group(df_group_weekday, 4)
weekly_sum = weekly_sum[['week_group', '去重订单数', '去重订单数环比', '进件数', '进件数环比', '预授权通过率', '预授权率环比差值', '出库', '出库环比', '进件出库率', '进件出库率环比差值', '订单出库率', '订单出库率环比差值']]
weekly_sum.set_index('week_group')

,去重订单数,去重订单数环比,进件数,进件数环比,预授权通过率,预授权率环比差值,出库,出库环比,进件出库率,进件出库率环比差值,订单出库率,订单出库率环比差值
week_group,,,,,,,,,,,,
20250627-20250703,46685,nan%,14463,nan%,30.98%,nan%,1357,nan%,9.38%,nan%,2.91%,nan%
20250704-20250710,51150,9.56%,17450,20.65%,34.12%,3.14%,1543,13.71%,8.84%,-0.54%,3.02%,0.11%


# 搜索

In [28]:

df_ss = df[df["归属渠道"]=="搜索渠道"]
df_ss2 = df2[df2["归属渠道"]=="搜索渠道"]

df_group_weekday_ss = all_models.data_group(df_ss, df_ss2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
weekly_sum_ss = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_ss, 4)
# weekly_sum_ss = weekly_sum_ss[['week_group', '去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('week_group')
weekly_sum_ss[['去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('去重订单数')

,订单占比,进件数,进件占比,出库,出库占比,进件出库转化率,订单出库率
去重订单数,,,,,,,
9412,20.16%,2871,19.85%,284,20.93%,9.89%,3.02%
9843,19.24%,3106,17.80%,317,20.54%,10.21%,3.22%


In [29]:
# 自然周
weekly_sum_ss7 = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_ss, 7)[['进件数', '出库']]

# 芝麻

In [30]:
# df_zm = df[df["归属渠道"]=="芝麻租物"]
# df_zm2 = df2[df2["归属渠道"]=="芝麻租物"]

# df_group_weekday_zm = all_models.data_group(df_zm, df_zm2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
# weekly_sum_zm = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_zm, 1)
# weekly_sum_zm = weekly_sum_zm[['week_group', '去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('week_group')
# # weekly_sum_zm[['week_group', '去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']]#.set_index('去重订单数')
# with pd.ExcelWriter('F:/需求/七月需求/上周二到本周一转化数据.xlsx', engine='xlsxwriter') as writer:
#     weekly_sum_zm.to_excel(writer, sheet_name='芝麻租物')
#     weekly_sum_ss.to_excel(writer, sheet_name='搜索渠道')

# 单人聊天

In [31]:
df_dr=df[df["来源渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df_dr2=df2[df2["来源渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]

df_group_weekday_dr = all_models.data_group(df_dr, df_dr2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]

# 芝麻租物

In [32]:
df_zm=df[df["归属渠道"]=="芝麻租物"]
df_zm2=df2[df2["归属渠道"]=="芝麻租物"]

df_group_weekday_zm = all_models.data_group(df_zm, df_zm2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
weekly_sum_zm = week_models.week_data_channel_group(df_group_weekday, df_group_weekday_zm, 4)
# weekly_sum_zm[['week_group', '去重订单数', '订单占比', '是否进件', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('去重订单数')
weekly_sum_zm[['去重订单数', '订单占比', '进件数', '进件占比', '出库', '出库占比', '进件出库转化率', '订单出库率']].set_index('去重订单数')

,订单占比,进件数,进件占比,出库,出库占比,进件出库转化率,订单出库率
去重订单数,,,,,,,
21989,47.10%,4714,32.59%,809,59.62%,17.16%,3.68%
22245,43.49%,5034,28.85%,865,56.06%,17.18%,3.89%


# 抖音

In [33]:
classify_id_list = [125,126,127,185]
df_dy=df[(df["归属渠道"]=="抖音渠道")&(df.classify_id.isin(classify_id_list))]
df_dy2=df2[(df2["归属渠道"]=="抖音渠道")&(df2.classify_id.isin(classify_id_list))]

df_group_weekday_dy = all_models.data_group(df_dy, df_dy2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]
df_group_weekday_dy = clean.continuous_dates(df_group_weekday_dy).reset_index(names=['下单日期']).set_index('下单日期')

# 支付宝直播

In [34]:
df_zfb = df[df['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb2 = df2[df2['activity_name'].str.contains('支付宝直播商品')==True]

df_group_weekday_df_zfb = all_models.data_group(df_zfb, df_zfb2, df_risk_examine, '下单日期')[['去重订单数', '进件数', '出库']]

# 总体剔除直播数据

In [35]:
weekly_sum_tc = df_group_weekday - df_group_weekday_dr - df_group_weekday_dy - df_group_weekday_df_zfb
weekly_sum_tc['出库'] =  df_group_weekday['出库'] - df_group_weekday_dr['出库'] - df_group_weekday_dy['出库'] - df_group_weekday_df_zfb['出库']
weekly_sum_tc_group = week_models.week_data_channel_group(df_group_weekday, weekly_sum_tc, 4)
# weekly_sum_tc_group[['week_group', '去重订单数', '是否进件', '出库', '预授权通过率', '进件出库转化率', '订单出库率']].set_index('去重订单数')
weekly_sum_tc_group[['去重订单数', '进件数', '出库', '预授权通过率', '进件出库转化率', '订单出库率']].set_index('去重订单数')

,进件数,出库,预授权通过率,进件出库转化率,订单出库率
去重订单数,,,,,
43390.0,13638.0,1348.0,31.43%,9.88%,3.11%
47849.0,16567.0,1536.0,34.62%,9.27%,3.21%


# 免审

In [36]:
df_ms = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]

df_group_ms = all_models.data_group(df_ms, df_ms2, df_risk_examine, '下单日期')[['进件数', '出库']].rename(columns={'进件数': '免审进件', '出库': '免审出库'})
df_group_ms['总体进件'] = df_group_weekday['进件数'][:-1]
df_group_ms['芝麻租物进件'] = df_group_weekday_zm['进件数'][:-1]

# 各渠道每日免审出库单数统计
df_group_ms['总体出库'] = df_group_weekday['出库'][:-1]
df_group_ms['芝麻租物出库'] = df_group_weekday_zm['出库'][:-1]

# 免审出库
dfck_ms = clean.drop_rejected_merchant(dfck)
dfdfck=dfck_ms[(dfck_ms.is_vip=='1')&(dfck_ms.status_result=='0')]
dfdfck_ms = pd.crosstab(dfdfck["下单日期"],dfdfck["归属渠道"],margins=True).reset_index()[['下单日期', '芝麻租物','搜索渠道']][:-1]
dfdfck_ms['下单日期'] = pd.to_datetime(dfdfck_ms['下单日期'])
dfdfck_ms.rename(columns={'芝麻租物': '芝麻租物免审出库', '搜索渠道': '搜索免审出库'}, inplace=True)
# 获取渠道免审进件
dfms = pd.crosstab(df_ms["下单日期"],df_ms["归属渠道"],margins=True).reset_index()[:-1]
dfms['下单日期']  = pd.to_datetime(dfms['下单日期'])
dfms = dfms[['下单日期', "芝麻租物","搜索渠道"]]

dfms.rename(columns={'芝麻租物': '芝麻租物免审进件', '搜索渠道': '搜索免审进件'}, inplace=True)
df_group_weekday_ms = pd.merge(df_group_ms, dfms, on='下单日期', how='outer')#[:-1]

# 关联数据
df_group_weekday_ms = pd.merge(df_group_weekday_ms, dfdfck_ms, on='下单日期', how='outer')
df_group_weekday_ms.set_index('下单日期', inplace=True)
# 进行周汇总
weekly_sum_ms = week_models.week_data_m_group(df_group_weekday_ms, 7, '免审')
weekly_sum_ms['搜索免审进件占比'] = weekly_sum_ms['搜索免审进件']/weekly_sum_ss7['进件数'] * 100
weekly_sum_ms['搜索免审进件占比'] = weekly_sum_ms['搜索免审进件占比'].apply(lambda x: f'{x:.2f}%')

weekly_sum_ms['搜索免审出库占比'] = weekly_sum_ms['搜索免审出库']/weekly_sum_ss7['出库'] * 100
weekly_sum_ms['搜索免审出库占比'] = weekly_sum_ms['搜索免审出库占比'].apply(lambda x: f'{x:.2f}%')

weekly_sum_ms[['week_group', '总体进件', '芝麻租物进件', '免审进件', '芝麻租物免审进件', '搜索免审进件', '免审进件占比', '芝麻租物免审进件占比', '搜索免审进件占比', '总体出库', '芝麻租物出库', '免审出库', '芝麻租物免审出库', '搜索免审出库', '免审转化率', '免审出库占比', '芝麻租物免审出库占比', '搜索免审出库占比']].set_index('week_group')

,总体进件,芝麻租物进件,免审进件,芝麻租物免审进件,搜索免审进件,免审进件占比,芝麻租物免审进件占比,搜索免审进件占比,总体出库,芝麻租物出库,免审出库,芝麻租物免审出库,搜索免审出库,免审转化率,免审出库占比,芝麻租物免审出库占比,搜索免审出库占比
week_group,,,,,,,,,,,,,,,,,
20250630-20250706,15948.0,5036.0,4319,2577,889,27.08%,51.17%,29.47%,1553.0,905.0,1527,902.0,318.0,35.36%,98.33%,99.67%,100.00%
20250707-20250713,17391.0,4848.0,4483,2519,915,25.78%,51.96%,30.40%,1413.0,755.0,1376,743.0,295.0,30.69%,97.38%,98.41%,100.00%


# 免审订单转化

In [37]:
df_msdd = df[(df.is_vip=='1')&(df.status_result=='0')]
df_msdd2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_group_mszh = all_models.data_group(df_msdd, df_msdd2, df_risk_examine, '下单日期')[['进件数', '机审强拒', '机审通过件',  '人审拒绝', '风控通过件', '客户取消','无法联系','出库前风控强拒','待审核', '出库']]

weekly_sum_mszh = week_models.week_data_m_group(df_group_mszh, 7, '免审订单')
weekly_sum_mszh[['week_group', '进件数', '人审拒绝', '客户取消', '出库', '待审核', '进件出库率', '取消率', '人审拒绝率', '出库前风控强拒', '出库前强拒比例', '无法联系', '无法联系占比']].set_index('week_group')
# weekly_sum_mszh = weekly_sum_mszh[['week_group', "进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].set_index('week_group')
# weekly_sum_mszh

,进件数,人审拒绝,客户取消,出库,待审核,进件出库率,取消率,人审拒绝率,出库前风控强拒,出库前强拒比例,无法联系,无法联系占比
week_group,,,,,,,,,,,,
20250630-20250706,4319,1779.0,525,1527,0,35.36%,12.16%,41.19%,169.0,3.91%,317,7.34%
20250707-20250713,4483,2115.0,447,1376,148,30.69%,9.97%,47.18%,161.0,3.59%,203,4.53%


# 免押订单转化

In [38]:
df_mydd1 = df[df["押金类型"]=="全免押"]
df_mydd2 = df2[df2["押金类型"]=="全免押"]

df_group_mydd = all_models.data_group(df_mydd1, df_mydd2, df_risk_examine, '下单日期')[['进件数', '机审强拒', '机审通过件',  '人审拒绝', '风控通过件', '客户取消','无法联系','出库前风控强拒','待审核', '出库']]
df_group_mydd['总进件'] = df_group_weekday['进件数']
myck = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)
df_group_mydd['出库'] = myck['全免押'][:-1]

weekly_sum_mydd = week_models.week_data_m_group(df_group_mydd, 7, '免审订单')
weekly_sum_mydd['免押进件占比'] = weekly_sum_mydd['进件数']/weekly_sum_mydd['总进件'] * 100
weekly_sum_mydd['免押进件占比'] = weekly_sum_mydd['免押进件占比'].apply(lambda x: f'{x:.2f}%')
weekly_sum_mydd[['week_group', '总进件', '免押进件占比', '进件数', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '待审核', "出库", '进件出库率', '取消率', '人审转化率', '人审拒绝率']].set_index('week_group')

,总进件,免押进件占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库,进件出库率,取消率,人审转化率,人审拒绝率
week_group,,,,,,,,,,,,,,,,
20250630-20250706,15948,97.91%,15614,10670,68.34%,5019.0,2229,2790.0,17.87%,584,0,1599.0,10.24%,3.74%,31.86%,14.28%
20250707-20250713,17391,98.00%,17044,11890,69.76%,5232.0,2675,2557.0,15.00%,490,156,1440.0,8.45%,2.87%,27.52%,15.69%


# 非免押订单转化

In [39]:
df_fmydd1 = df[df["押金类型"]=="部分免押"]
df_fmydd2 = df2[df2["押金类型"]=="部分免押"]

df_group_fmydd = all_models.data_group(df_fmydd1, df_fmydd2, df_risk_examine, '下单日期')[['进件数', '机审强拒', '机审通过件',  '人审拒绝', '风控通过件', '客户取消','无法联系','出库前风控强拒','待审核', '出库']]
df_group_fmydd['总进件'] = df_group_weekday['进件数']
weekly_sum_fmydd = week_models.week_data_m_group(df_group_fmydd, 7, '免审订单')
weekly_sum_fmydd['非免押进件占比'] = weekly_sum_fmydd['进件数']/weekly_sum_fmydd['总进件'] * 100
weekly_sum_fmydd['非免押进件占比'] = weekly_sum_fmydd['非免押进件占比'].apply(lambda x: f'{x:.2f}%')
weekly_sum_fmydd[['week_group', '总进件', '非免押进件占比', '进件数', '机审强拒', '强拒比例', '机审通过件', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '待审核', "出库", '进件出库率', '取消率', '人审转化率', '人审拒绝率']].set_index('week_group')

,总进件,非免押进件占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库,进件出库率,取消率,人审转化率,人审拒绝率
week_group,,,,,,,,,,,,,,,,
20250630-20250706,15948,2.09%,334,326.0,97.60%,8.0,2,6.0,1.80%,0,0,1,0.30%,0.00%,12.50%,0.60%
20250707-20250713,17391,2.00%,347,340.0,97.98%,7.0,0,7.0,2.02%,2,0,1,0.29%,0.58%,14.29%,0.00%


# 商家周数据

## 澄心优租

In [40]:
week_cxyz = week_models.week_data_group_merchant(df_merchant, '澄心优租', '下单日期', 7)
week_cxyz[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]].set_index('week_group')

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,人审拒绝,人审拒绝率,待审核,出库,进件出库率,订单出库率
week_group,,,,,,,,,,,,
20250630-20250706,48.0,0.0,0.00%,9.0,18.75%,9.0,8.0,88.89%,0.0,0.0,0.00%,0.00%
20250707-20250713,37.0,1.0,2.70%,6.0,16.22%,6.0,2.0,33.33%,0.0,0.0,0.00%,0.00%


## 优优2店

In [41]:
# week_yy2 = week_models.week_data_group_merchant(df_merchant, '优优2店', '下单日期', 7, '商家')
# week_yy2[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 北京海鸟窝科技有限公司

In [42]:
week_hnw = week_models.week_data_group_merchant(df_merchant, '北京海鸟窝科技有限公司', '下单日期', 7)
week_hnw[['week_group',"去重订单数","进件数","预授权通过率","出库","进件出库率"]].set_index('week_group')


,去重订单数,进件数,预授权通过率,出库,进件出库率
week_group,,,,,
20250630-20250706,2.0,0.0,0.00%,0.0,nan%
20250707-20250713,5.0,2.0,40.00%,0.0,0.00%


## 租着用电脑数码

In [43]:
week_zzy = week_models.week_data_group_merchant(df_merchant, '租着用电脑数码', '下单日期', 7)
week_zzy[['week_group',"去重订单数","进件数","预授权通过率","出库","进件出库率"]].set_index('week_group')


,去重订单数,进件数,预授权通过率,出库,进件出库率
week_group,,,,,
20250630-20250706,204,85,41.67%,4,4.71%
20250707-20250713,162,59,36.42%,4,6.78%


## 趣智数码

In [44]:
week_zqsm = week_models.week_data_group_merchant(df_merchant, '趣智数码', '下单日期', 7)
week_zqsm[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
week_group,,,,,,,,,,,,,,
20250630-20250706,26.0,0.0,0.00%,12.0,46.15%,12.0,1.0,3.0,25.00%,0.0,2.0,16.67%,8.33%,7.69%
20250707-20250713,28.0,0.0,0.00%,9.0,32.14%,9.0,1.0,0.0,0.00%,0.0,3.0,33.33%,11.11%,10.71%


## 汇客好租

In [45]:
week_hkhz = week_models.week_data_group_merchant(df_merchant, '汇客好租', '下单日期', 7)
week_hkhz[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
week_group,,,,,,,,,,,,,,
20250630-20250706,159,34,21.38%,71,44.65%,71,2,23,32.39%,0,13,18.31%,2.82%,8.18%
20250707-20250713,290,44,15.17%,136,46.90%,136,3,47,34.56%,0,20,14.71%,2.21%,6.90%


## 小蚂蚁租机

In [46]:
# week_xmy = week_models.week_data_group_merchant(df_merchant, '小蚂蚁租机', '下单日期', 7)
# week_xmy[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 乙辉数码

In [47]:
# week_yhsm = week_models.week_data_group_merchant(df_merchant, '乙辉数码', '下单日期', 7)
# week_yhsm[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


## 兴鑫兴通讯

In [48]:
# week_xxx = week_models.week_data_group_merchant(df_merchant, '兴鑫兴通讯', '下单日期', 7)
# week_xxx[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


## 瓜子笔记本电脑

In [49]:
# week_gz = week_models.week_data_group_merchant(df_merchant, '呱子笔记本电脑', '下单日期', 7)
# week_gz[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 南京聚格网络科技

In [50]:
# week_jg = week_models.week_data_group_merchant(df_merchant, '南京聚格网络科技', '下单日期', 7)
# week_jg[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 人人享租

In [51]:
week_rrxz = week_models.week_data_group_merchant(df_merchant, '人人享租', '下单日期', 7)
week_rrxz[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
week_group,,,,,,,,,,,,,,
20250630-20250706,2141,0,0.00%,2140,99.95%,160,1,78,3.64%,0,78,3.64%,0.05%,3.64%
20250707-20250713,1995,0,0.00%,1994,99.95%,141,3,72,3.61%,0,66,3.31%,0.15%,3.31%


## 崇胜数码

In [52]:
# week_cssm = week_models.week_data_group_merchant(df_merchant[df_merchant.下单日期>='2025-05-20'], '崇胜数码', '下单日期', 7)
# week_cssm[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


## 喜卓林租机

In [53]:
week_czl = week_models.week_data_group_merchant(df_merchant[df_merchant.下单日期>='2025-05-20'], '喜卓灵租机', '下单日期', 7)
week_czl[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
week_group,,,,,,,,,,,,,,
20250630-20250706,365,0,0.00%,365,100.00%,27,1,17,4.66%,0,9,2.47%,0.27%,2.47%
20250707-20250713,378,0,0.00%,378,100.00%,49,0,39,10.32%,1,9,2.38%,0.00%,2.38%


## 喜卓灵新租机

In [54]:
week_xzlx = week_models.week_data_group_merchant(df_merchant[df_merchant.下单日期>='2025-05-20'], '喜卓灵新租机', '下单日期', 7)
week_xzlx[['week_group', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]].set_index('week_group')


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
week_group,,,,,,,,,,,,,,
20250630-20250706,133,0,0.00%,133,100.00%,14,2,4,3.01%,0,8,6.02%,1.50%,6.02%
20250707-20250713,334,0,0.00%,334,100.00%,13,1,5,1.50%,0,6,1.80%,0.30%,1.80%
